In [15]:
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [16]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [17]:
Base = automap_base()

Base.prepare(engine, reflect=True)

In [18]:
Base.classes.keys()

['measurement', 'station']

In [19]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [20]:
app = Flask(__name__)

In [21]:
@app.route("/")  
def home():  
    return (f"Hello, welcome to Surfs Up Climate API!<br>"
            f"All Available Routes<br>"
            f"/api/v1.0/precipitation<br>"
            f"/api/v1.0/stations<br>"
            f"/api/v1.0/tobs<br>"
            f"/api/v1.0/<start><br>"
            f"/api/v1.0/<start>/<end><br>")

In [22]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    results = session.query(Measurement).all()
    date_prcp = []
    for result in results:
        date_prcp_dict = {result.date, result.prcp}
        data_prcp.append(data_prcp_dict)
    return jsonify(data_prcp)

In [23]:
@app.route("/api/v1.0/stations")
def stations():
    results = session.query(Station.station).all()
    stations_list = list(np.ravel(results))
    return jsonify(stations_list)

In [24]:
@app.route("/api/v1.0/tobs")
def tobs():
    last_twelve_months = dt.date(2017,8,23) - dt.timedelta(days=365)
    Active_station = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).first()
    Active_station_tobs = session.query(Measurement.station, Measurement.tobs).filter(Measurement.station == Active_station).filter(Measurement.date >= last_twelve_months).all()
    Active_station_tobs_list = list(np.ravel(Active_station_tobs))
    return jsonify(Active_station_tobs_list)

In [33]:
@app.route("/api/v1.0/<start>")
def start(start):
    Start_date = dt.datetime.strptime(start,"%y-%m-%d")
    Start_temp_range = session.query(func.min(Measurement.tobs),func.avg(Measurement.tobs),func.max(Measurement.tobs)).filter(Measurement.date >= Start_date).all()
    Start_list = list(np.ravel(Start_temp_range))
    return jsonify(Start_list)

In [35]:
@app.route("/api/v1.0/<start>/<end>")
def startandend(start,end):
    Start_date = dt.datetime.strptime(start,"%y-%m-%d")
    End_date = dt.datetime.strptime(end,"%y-%m-%d")
    Start_end_range = session.query(func.min(Measurement.tobs),func.avg(Measurement.tobs),func.max(Measurement.tobs)).filter(Measurement.date.between(Start_date,End_date)).all()
    Start_end_list = list(np.ravel(Start_end_range))
    return jsonify(Start_end_list)